# 02 - MPS Performance and Scaling: Quantifying the Quantum Advantage

This notebook expands upon the concepts introduced in `examples/06_mps_entanglement_demo.py` by demonstrating the practical performance gains of the Matrix Product State (MPS) backend for large, low-entanglement circuits. 

We will compare the execution time of the MPS backend against a forced run on a dense state vector backend (Qiskit's simulator) as the number of qubits increases, showcasing the polynomial speedup of MPS in this regime.

In [ ]:
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit

from ariadne.router import BackendType, simulate

# Suppress Qiskit warnings during simulation
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 1. Low Entanglement Circuit Generator

We define a circuit that maintains low entanglement, ensuring the MPS bond dimension (D) grows slowly (e.g., polynomially or linearly) with the number of qubits (N). This is the 'sweet spot' for MPS.

In [ ]:
def create_low_entanglement_circuit(n_qubits: int) -> QuantumCircuit:
    """Creates a circuit with low entanglement (e.g., a simple 1D nearest-neighbor structure)."""
    qc = QuantumCircuit(n_qubits)

    # Initial layer of H gates
    qc.h(range(n_qubits))

    # Shallow layer of nearest-neighbor CNOTs
    for i in range(0, n_qubits - 1, 2):
        qc.cx(i, i + 1)

    # Repeat a few times to increase depth slightly without maximizing entanglement
    for _ in range(2):
        for i in range(1, n_qubits - 1, 2):
            qc.cx(i, i + 1)
        qc.barrier()

    qc.measure_all()
    return qc

## 2. Performance Comparison

We simulate the circuit across a range of qubit counts, forcing the use of the MPS backend and the Qiskit dense state vector backend for direct comparison.

In [ ]:
# Test range: up to 20-22 qubits, where SV simulation becomes noticeably slow
N_QUBITS_RANGE = np.arange(8, 22, 2)
mps_times = []
sv_times = []

print("Starting performance comparison...")

for N in N_QUBITS_RANGE:
    qc = create_low_entanglement_circuit(N)
    shots = 1024

    print(f"\nSimulating N={N} qubits...")

    # --- 1. MPS Backend (Expected Polynomial Scaling) ---
    try:
        # Force MPS backend
        start_mps = time.perf_counter()
        result_mps = simulate(qc, shots=shots, backend=BackendType.MPS.value)
        end_mps = time.perf_counter()
        mps_time = end_mps - start_mps
        mps_times.append(mps_time)
        print(f"  MPS Time: {mps_time:.4f}s (Backend: {result_mps.backend_used.value})")
    except Exception as e:
        print(f"  MPS simulation failed for N={N}: {e}")
        mps_times.append(np.nan)

    # --- 2. Dense State Vector Backend (Expected Exponential Scaling) ---
    try:
        # Force Qiskit's dense state vector simulator for comparison (using 1 shot for speed)
        start_sv = time.perf_counter()
        result_sv = simulate(qc, shots=1, backend=BackendType.QISKIT.value)
        end_sv = time.perf_counter()
        sv_time = end_sv - start_sv
        sv_times.append(sv_time)
        print(f"  SV Time: {sv_time:.4f}s (Backend: {result_sv.backend_used.value})")
    except Exception as e:
        print(f"  SV simulation failed for N={N}: {e}")
        sv_times.append(np.nan)

print("\nComparison complete.")

## 3. Visualization and Quantification

The plot below clearly illustrates the difference in scaling. The dense state vector simulation time grows exponentially (a straight line on a log scale), while the MPS simulation time grows much slower (polynomial), leading to massive speedups for larger circuits.

In [ ]:
# Filter out NaNs for plotting
valid_indices = ~np.isnan(mps_times) & ~np.isnan(sv_times)
N_valid = N_QUBITS_RANGE[valid_indices]
mps_valid = np.array(mps_times)[valid_indices]
sv_valid = np.array(sv_times)[valid_indices]

plt.figure(figsize=(10, 6))

# Plot MPS performance (polynomial)
plt.plot(N_valid, mps_valid, "go-", label="MPS Backend (Polynomial Scaling)")

# Plot State Vector performance (exponential)
plt.plot(N_valid, sv_valid, "r^-", label="Dense State Vector (Exponential Scaling)")

plt.yscale("log")
plt.title("Performance Scaling: MPS vs. Dense State Vector for Low-Entanglement Circuits")
plt.xlabel("Number of Qubits (N)")
plt.ylabel("Execution Time (seconds) [Log Scale]")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.show()

# Quantification
if N_valid.size > 0:
    max_n = N_valid[-1]
    mps_time_max = mps_valid[-1]
    sv_time_max = sv_valid[-1]

    if mps_time_max > 0 and sv_time_max > 0:
        speedup = sv_time_max / mps_time_max
        print(f"\n--- Performance Quantification (N={max_n} Qubits) ---")
        print(f"Dense State Vector Time: {sv_time_max:.4f}s")
        print(f"MPS Backend Time: {mps_time_max:.4f}s")
        print(f"Speedup (SV/MPS): {speedup:.2f}x")
        print("\nConclusion: The MPS backend demonstrates a significant speedup for low-entanglement circuits,")
        print(
            "confirming the theoretical polynomial scaling advantage over the exponentially scaling dense state vector method."
        )